# EMA Cross Strategy Example

This notebook demonstrates how to use the C++ Backtest Engine with a simple EMA Crossover strategy.

In [1]:
import sys
sys.path.append('../python')
from backtester import Backtester, Strategy, Bar, Side
import random
import time

In [2]:
class EmaCrossStrategy(Strategy):
    def __init__(self, engine, fast_period=10, slow_period=20):
        super().__init__(engine)
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.prices = []
        
    def on_bar(self, bar: Bar):
        self.prices.append(bar.close)
        
        if len(self.prices) < self.slow_period:
            return
            
        # Calculate EMAs (using GPU for demonstration if list is long, but here simple CPU)
        # For demo, we just use simple average of last N
        fast_ma = sum(self.prices[-self.fast_period:]) / self.fast_period
        slow_ma = sum(self.prices[-self.slow_period:]) / self.slow_period
        
        # Trading Logic
        pos_qty = self.engine.get_portfolio().get_position_quantity(bar.instrument_id)
        
        if fast_ma > slow_ma and pos_qty <= 0:
            print(f"Buy at {bar.close}")
            self.buy(bar.instrument_id, 100)
        elif fast_ma < slow_ma and pos_qty >= 0:
            print(f"Sell at {bar.close}")
            self.sell(bar.instrument_id, 100)

In [3]:
# Run Simulation
backtester = Backtester()
strategy = backtester.add_strategy(EmaCrossStrategy)

# Generate dummy data
bars = []
price = 100.0
for i in range(1000):
    price += (random.random() - 0.5)
    bar = Bar()
    bar.timestamp = i * 60 * 1000000000 # nanoseconds
    bar.instrument_id = 1
    bar.close = price
    bar.open = price
    bar.high = price
    bar.low = price
    bar.volume = 1000
    bars.append(bar)

backtester.add_data(bars)

start = time.time()
backtester.run()
end = time.time()

print(f"Simulation took {end - start:.4f}s")
print(f"Final Equity: {backtester.get_portfolio().get_equity()}")

Buy at 101.50448400210615
Sell at 102.4642301458931
Sell at 102.37144761921645
Sell at 102.19610790724161
Sell at 102.34922379066914
Sell at 102.64107342780997
Sell at 102.5209438449992
Sell at 102.07976865818301
Sell at 101.94577852861804
Sell at 101.89302158613482
Sell at 101.70472717483149
Sell at 101.87356338085517
Sell at 101.4952547889095
Sell at 101.49569530881541
Sell at 101.4002850481503
Sell at 101.33880363744858
Sell at 101.57922248266934
Sell at 101.57659172844544
Sell at 101.19554592785992
Sell at 101.24652951387789
Sell at 101.01567040658824
Sell at 100.9400155947983
Sell at 100.92667731669385
Sell at 101.00296659899581
Sell at 100.99883376560456
Sell at 100.90839982976323
Sell at 100.45619805884559
Sell at 100.29209712781336
Sell at 100.05583711181973
Sell at 100.1849360412394
Sell at 100.16037622803921
Sell at 100.63393978165136
Sell at 100.58126355242538
Sell at 100.8583177557326
Sell at 101.20252744482536
Sell at 100.9517219445821
Sell at 101.35320236191971
Sell at 10